### Data Preprocessing

In [1]:
import pandas as pd
import numpy as np
import gensim
from gensim.models import Word2Vec
import itertools
from collections import Counter

from MovieClass import MovieClass, MultiMovieClass
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

In [87]:
#data selection
df = pd.read_csv('data_raw/ratings_small.csv')
df = df.loc[df.rating>=4.0]
#get number of movies watched by each userId
df_n = df['userId'].value_counts().to_frame()
df_n.columns = ['ntotal']
df_n['userId'] = df_n.index
# print(df_n) eliminate the data <21
df = df.merge(df_n, on = 'userId')
df = df.loc[df.ntotal>=30]

#sort by rating and timestap
df = df.sort_values(['userId','rating'],ascending=[1,0])
df = df.sort_values(['userId','timestamp'],ascending=[1,1])

df = df[['userId', 'movieId']]
df['count'] = df.groupby('userId').cumcount()
df.to_csv('data/users_data.csv')
df

,userId,movieId,count
9,2,150,0
33,2,590,1
34,2,592,2
17,2,296,3
10,2,153,4
...,...,...,...
51501,671,1035,79
51565,671,5995,80
51541,671,3271,81
51567,671,6365,82


In [89]:
#split and save inputs and truth ground 
df_X = df[df['count']<20]
df_Y = df[df['count']>=20]
df_X = df_X[['userId', 'movieId']]
df_Y = df_Y[['userId', 'movieId']]

# df_X.to_csv('data/users_history.csv')
# df_Y.to_csv('data/ground_truth.csv')

# Step-1: recall 

### "Movie2Movie" Co-visitation Matrix - Rating Weighted

In [13]:
df = pd.read_csv('data_raw/ratings_small.csv').drop('timestamp', axis=1)
df = pd.merge(df, df, on ='userId')
df = df.loc[df.movieId_x != df.movieId_y]
df['wgt'] = df.rating_y / 5
df = df[['movieId_x','movieId_y','wgt']]
df = df.groupby(['movieId_x','movieId_y']).wgt.sum()
df = df.reset_index()
df = df.sort_values(['movieId_x','wgt'], ascending=[True, False])
df = df.reset_index(drop = True)
df['n'] = df.groupby('movieId_x').movieId_y.cumcount()
df = df.loc[df.n <20].drop('n', axis=1)
df.to_csv('data/Movie2Movie.csv')

### Most Popular Movies

In [63]:
#from full dataset
df = pd.read_csv('data_raw/ratings.csv')
top_view_movies = df.movieId.value_counts().index.values[:50]
top_view_movies

array([  356,   318,   296,   593,  2571,   260,   480,   527,   110,
           1,  1210,   589,  1196,  2959,  1198,    50,  2858,   150,
         780,   858,  4993,   457,  1270,   592,    32,    47,   608,
        5952,  7153,   590,  2028,   588,   380,  2762,   377,  3578,
         364,   344,   648,  4306,  1721,  1580,   595,  1197,  4226,
         165,  1240,  1193,  1291, 58559])

In [64]:
df = df.movieId.value_counts().to_frame()
df = df.reset_index()
df.columns = ['movieId','ratingNum']
df['populairty'] = df['ratingNum'].apply(lambda x: int(x/9000))
df_pop = df[['movieId','populairty']]
df_pop.to_csv('data/popularity.csv')
df_pop.head()

,movieId,populairty
0,356,10
1,318,10
2,296,9
3,593,9
4,2571,8


### Same director and cast 

In [53]:
# df3 = pd.read_csv('data_raw/credits.csv')
# directors = []
# casts = []
# for i in range(len(df3)):
#     for member in eval(df3.iloc[i].values[1]):
#         directors.append(member['id'])
    
#     list = []
#     for member in eval(df3.iloc[i].values[0]):
#         list.append(member['id'])   
#     list = list[:10] if len(list) > 10 else list+[-1]*(10-len(list))
#     casts.append(list)

# df3['director'] = pd.DataFrame(directors)
# df_cast = pd.DataFrame(casts)
# df3 = pd.concat([df3, df_cast], axis=1)
# df3.rename(columns={'id': 'tmdbid', 0: 'cast1',1: 'cast2',2: 'cast3',3: 'cast4',4: 'cast5',5: 'cast6',6: 'cast7',7: 'cast8',8: 'cast9',9: 'cast10'}, inplace=True)
# df3 = df3[['tmdbid','director', 'cast1', 'cast2', 'cast3', 'cast4', 'cast5', 'cast6', 'cast7', 'cast8', 'cast9', 'cast10']]
# df3.to_csv('data/movie_director_casts.csv')
# df3.head()

,tmdbid,director,cast1,cast2,cast3,cast4,cast5,cast6,cast7,cast8,cast9,cast10
0,862,7879,31,12898,7167,12899,12900,7907,8873,1116442,12901,12133
1,8844,12891,2157,8537,205,145151,5149,10739,58563,1276,46530,56523
2,15602,7,6837,3151,13567,16757,589,16523,7166,-1,-1,-1
3,31357,12892,8851,9780,18284,51359,66804,352,87118,34,1276777,10814
4,11862,12893,67773,3092,519,70696,59222,18793,14592,20906,54348,209


# Step-2: Rerank

### word2vec

In [142]:
#read keywords file
df1 = pd.read_csv('data_raw/keywords.csv')

wv_train_data = []
for i in range(len(df1)):
    key_list = []
    if len(df1.iloc[i].values):
        for key in eval(df1.iloc[i].values[1]):
            key_list.append(key['name'])
    if key_list:
        wv_train_data.append(key_list) 

model = Word2Vec(sentences=wv_train_data, vector_size=32, min_count=0, workers=5, window=5, epochs=30)
model.save("word2vec.model")

In [90]:
cosine_similarity(model.wv['man'].reshape(1,-1), model.wv['vampire'].reshape(1,-1))

array([[0.3885254]], dtype=float32)

### load data

In [47]:
def df_covisitation_to_dict(df):
    return df.groupby('movieId_x').movieId_y.apply(list).to_dict()

def df_user_to_dict(df):
    return df.groupby('userId').movieId.apply(list).to_dict()


df_cov = pd.read_csv('data/Movie2Movie.csv')
df_X = pd.read_csv('data/users_history.csv')
df_y = pd.read_csv('data/ground_truth.csv')
df_dc = pd.read_csv('data/movie_director_casts.csv')
df_pop = pd.read_csv('data/popularity.csv')
top_20_movies = df_covisitation_to_dict(df_cov)
X_dic = df_user_to_dict(df_X)
gt_dic = df_user_to_dict(df_y)
model_wv = Word2Vec.load("word2vec.model")

### Bulid features

feature1: rank of movie2movie, replace None by mean  

feature2: similarity of user and the movie  

feature3: user vector covarience  

feature4: movie popularity 

